In [1]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import pandana as pdn
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import warnings
import os
import json

warnings.filterwarnings("ignore")

In [2]:
main_path = '/app'

In [3]:
pois = gpd.read_file(f'{main_path}/data/input/amenities_costanera')

In [4]:
def load_pandana_network_graph():
    graph = ox.graph_from_place(
        "Concepción, Chile",
        network_type="walk",
        simplify=False,
    )

    nodes_df, edges_df = ox.graph_to_gdfs(graph)
    nodes_df = nodes_df.rename(columns={'x': 'lon', 'y': 'lat'})
    nodes_df['id'] = nodes_df.index
    edges_df['from'] = edges_df.reset_index()['u'].values
    edges_df['to'] = edges_df.reset_index()['v'].values

    net = pdn.Network(
        nodes_df['lon'],
        nodes_df['lat'],
        edges_df['from'],
        edges_df['to'],
        edges_df[['length']]
    )
    return net

download = False

if(download):
    net = load_pandana_network_graph()
    net.save_hdf5(f'{main_path}/data/output/walking_actual_graph_conce.h5')
else:
    net = pdn.Network.from_hdf5(f'{main_path}/data/output/walking_actual_graph_conce.h5')

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 37494
Setting CH edge vector of size 82736
Range graph removed 82736 edges of 165472
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [5]:
grid = gpd.read_file(f'{main_path}/data/input/grilla')

In [6]:
# Agrega el centroide a la grilla y los puntos (x,y)
grid['centroid'] = grid['geometry'].centroid
grid['x'] = grid['centroid'].x
grid['y'] = grid['centroid'].y

# Agrega los puntos (x,y) de los pois
pois['x'] = pois['geometry'].x
pois['y'] = pois['geometry'].y

In [7]:
grid['nearest_node'] = net.get_node_ids(grid['x'], grid['y'])
pois['nearest_node'] = net.get_node_ids(pois['x'], pois['y'])

In [8]:
pois.to_file(f'{main_path}/data/output/multiprocess_pois')

Método utilizando un solo procesador

In [9]:
n_rank = 3
output_cols = ['grid_id', 'rank', 'origin', 'destination', 'path_lengths', 'Category', 'Subcategor', 'geometry']

categories = pois['Category'].unique()
df_output = []

for category in categories:
    mask = pois['Category']==category
    destination_nodes = pois.loc[mask, 'nearest_node']
    for index, row in grid.iterrows():
        origin_node = grid.loc[index, 'nearest_node']
        origin_nodes = [origin_node]*len(destination_nodes)
        shortest_path_lengths = net.shortest_path_lengths(origin_nodes, destination_nodes)
        df = pd.DataFrame.from_dict({
            'origin': origin_nodes,
            'destination': destination_nodes,
            'path_lengths': shortest_path_lengths,
        })
        df.sort_values(by='path_lengths', inplace=True)
        df = df.head(n_rank)
        df['grid_id'] = grid.loc[index, 'grid_id']
        df.reset_index(inplace=True, drop=True)
        df['rank'] = df.index + 1
        df = pd.merge(df, pois[mask].rename(columns={'nearest_node': 'destination'}), on='destination')

        df_output.append(df[output_cols])

df_output = pd.concat(df_output)

In [ ]:
df_output

NameError: name 'df_output' is not defined